[![Azure Notebooks](https://notebooks.azure.com/launch.png)](https://notebooks.azure.com/import/gh/microsoft/AI-Utilities)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/microsoft/AI-Utilities/deep_learning_2?filepath=notebooks%2Fai-deep-realtime-score.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/microsoft/AI-Utilities/blob/deep_learning_2/notebooks/ai-deep-realtime-score.ipynb)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Deploy-Solution" data-toc-modified-id="Deploy-Solution-1">Deploy Solution</a></span><ul class="toc-item"><li><span><a href="#Create-Configuration" data-toc-modified-id="Create-Configuration-1.1">Create Configuration</a></span></li><li><span><a href="#Deploy-to-Azure-Kubernetes-Service-with-Azure-ML" data-toc-modified-id="Deploy-to-Azure-Kubernetes-Service-with-Azure-ML-1.2">Deploy to Azure Kubernetes Service with Azure ML</a></span></li></ul></li><li><span><a href="#Deploy-Services" data-toc-modified-id="Deploy-Services-2">Deploy Services</a></span><ul class="toc-item"><li><span><a href="#Machine-Learning-Studio" data-toc-modified-id="Machine-Learning-Studio-2.1">Machine Learning Studio</a></span></li><li><span><a href="#Kubernetes" data-toc-modified-id="Kubernetes-2.2">Kubernetes</a></span></li><li><span><a href="#Application-Insights" data-toc-modified-id="Application-Insights-2.3">Application Insights</a></span><ul class="toc-item"><li><span><a href="#Main" data-toc-modified-id="Main-2.3.1">Main</a></span></li><li><span><a href="#Availability" data-toc-modified-id="Availability-2.3.2">Availability</a></span></li><li><span><a href="#Performance-Dashboard" data-toc-modified-id="Performance-Dashboard-2.3.3">Performance Dashboard</a></span></li><li><span><a href="#Load-Test" data-toc-modified-id="Load-Test-2.3.4">Load Test</a></span></li></ul></li></ul></li></ul></div>

## Deploy Solution

### Create Configuration

In [9]:
import os
from azure_utils.machine_learning.factories.realtime_factory import RealTimeFactory
from azure_utils.machine_learning.models.training_arg_parsers import default_response
from azure_utils.machine_learning.ai_workspace import  DeepRealtimeScore
from azure_utils.notebook_widgets.notebook_configuration_widget import get_configuration_widget


project_configuration = "notebook_project.yml"

In [ ]:
configuration_widget = get_configuration_widget(project_configuration)

In [11]:
configuration_widget

In [20]:
inspect.getsource(RealTimeFactory)

'class RealTimeFactory:\n    def train(self, args):\n        raise NotImplementedError\n\n    def score_init(self):\n        raise NotImplementedError\n\n    @rawhttp\n    def score_run(self, request):\n        raise NotImplementedError\n'

In [13]:
class RealTimeDeepFactory(RealTimeFactory):
    def train(self, args):
        import os
        from azure_utils.samples.deep_rts_samples import ResNet152
        
        resnet_152 = ResNet152.create_model(weights="imagenet")
        resnet_152.save_model(os.path.join(args.outputs, args.model))

    def score_init(self):
        from azure_utils.samples.deep_rts_samples import ResNet152
        self.resnet_152 = ResNet152.load_model()

    def score_run(self, request):
        from azure_utils.samples.deep_rts_samples import ResNet152
        from azure_utils.machine_learning.models.training_arg_parsers import default_response
        """ Make a prediction based on the data passed in using the preloaded model"""
        if request.method == 'POST':
            return self.resnet_152.predict(request)
        return default_response(request)

    def __init__(self):
        self.resnet_152 = None


In [34]:
file = inspect.getsource(RealTimeFactory)
file = file.replace('def train(self, args):\n        raise NotImplementedError\n', inspect.getsource(RealTimeDeepFactory.train))
file = file.replace('def score_init(self):\n        raise NotImplementedError\n', inspect.getsource(RealTimeDeepFactory.score_init))
file = file.replace('@rawhttp\n    def score_run(self, request):\n        raise NotImplementedError\n', inspect.getsource(RealTimeDeepFactory.score_run))
file = file.replace('def __init__(self):\n        raise NotImplementedError\n', inspect.getsource(RealTimeDeepFactory.__init__))
file

'class RealTimeFactory:\n\n        def __init__(self):\n        self.resnet_152 = None\n\n        def train(self, args):\n        import os\n        from azure_utils.samples.deep_rts_samples import ResNet152\n        \n        resnet_152 = ResNet152.create_model(weights="imagenet")\n        resnet_152.save_model(os.path.join(args.outputs, args.model))\n\n        def score_init(self):\n        from azure_utils.samples.deep_rts_samples import ResNet152\n        self.resnet_152 = ResNet152.load_model()\n\n        def score_run(self, request):\n        from azure_utils.samples.deep_rts_samples import ResNet152\n        """ Make a prediction based on the data passed in using the preloaded model"""\n        if request.method == \'POST\':\n            return self.resnet_152.predict(request)\n        return default_response(request)\n'